# Overjeno šifriranje

Cilji laboratorijske vaje so sledeči:
- TODO

In [1]:
import os
import time
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, hmac, padding
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [2]:
def fernet_gen_key():
    return base64.urlsafe_b64encode(os.urandom(32))

In [3]:
def fernet_encrypt(key, message):
    # unpack and decode keys
    k_mac, k_enc = base64.urlsafe_b64decode(key)[:16], base64.urlsafe_b64decode(key)[16:]    
    
    version = (128).to_bytes(1, byteorder='big')
    timestamp = int(time.time()).to_bytes(8, byteorder='big')
    iv = os.urandom(16)
            
    # padding
    padder = padding.PKCS7(128).padder() # CBC padding
    padded_msg = padder.update(message) + padder.finalize()
    
    # encryption
    enc = Cipher(algorithms.AES(k_enc), modes.CBC(iv)).encryptor() # AES-CBC
    ct = enc.update(padded_msg) + enc.finalize()
    
    # mac
    h = hmac.HMAC(k_mac, hashes.SHA256()) # hmac
    h.update(version + timestamp + iv + ct)
    tag = h.finalize()
    
    # token
    return base64.urlsafe_b64encode(version + timestamp + iv + ct + tag)    

In [4]:
def fernet_decrypt(key, token, ttl=None):
    # unpack and decode keys
    k_mac, k_enc = base64.urlsafe_b64decode(key)[:16], base64.urlsafe_b64decode(key)[16:]
    
    # token
    token = base64.urlsafe_b64decode(token)
    
    version = token[0]
    assert version == 128, "Invalid version"
    
    timestamp = int.from_bytes(token[1:9], byteorder='big') 
    if ttl:
        assert int(time.time()) - timestamp > ttl, "Maximum age expired"
    
    # mac
    h = hmac.HMAC(k_mac, hashes.SHA256()) # hmac
    h.update(token[:-32])
    h.verify(token[-32:])
     
    # decryption
    iv = token[9:25]
    ct = token[25:-32]
    dec = Cipher(algorithms.AES(k_enc), modes.CBC(iv)).decryptor() # AES-CBC
    pt = dec.update(ct) + dec.finalize()
    
    # remove pad
    unpadder = padding.PKCS7(128).unpadder() # CBC padding
    msg = unpadder.update(pt) + unpadder.finalize()
    return msg

In [5]:
key = fernet_gen_key()
ct = fernet_encrypt(key, b"Hello World! A very long message, surely more than a single block.")
fernet_decrypt(key, ct)

b'Hello World! A very long message, surely more than a single block.'

In [6]:
from cryptography.fernet import Fernet

In [9]:
def load_java():
    with open("../data/fernet-java.key", "rb") as h:
        key = h.read()

    with open("../data/fernet-java.ct", "rb") as h:
        ct = h.read()
        
    return key, ct

In [10]:
key, ct = load_java()
f = Fernet(key)
f.decrypt(ct).decode("utf8")

'Hello Wold! Tole je primer sporočila.'

In [14]:
key = AESGCM.generate_key(bit_length=128)
key.hex()

'e5777ed7a740e7ba224dad00125c3482'

In [15]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [18]:
# Salts should be randomly generated
salt = os.urandom(16)
# derive
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=480000,
)
key = kdf.derive(b"hunter2")
key

b'js\xec\x00<G > \xde\x8bu\xaa\x19\xf8t\xc4\xf2\x82\xdd\x9d\xac=\xde\x115>\x85/K`\x84'